In [2]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached regex-2024.11.6-cp313-cp313-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp313-cp313-win_amd64.whl (273 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ISHAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import pandas as pd 
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ISHAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
news_data=pd.read_csv(r"D:\Ishan\ProCoder_KrutiVerse_Project\Data\fake_and_real_news.csv")

In [ ]:
news_data.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [7]:
news_data.shape

(9900, 2)

In [9]:
news_data.isnull().sum()

Text     0
label    0
dtype: int64

In [15]:
from nltk.corpus import stopwords

In [16]:
# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [ ]:
def preprocessing_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'<.*?>', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove numbers and special characters
    words = text.split()  # Tokenize text
    
    # Remove stopwords and short words
    words = [word for word in words if word not in stop_words and len(word) > 3]

    # Apply stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)  # Return string instead of list

In [18]:
stemmer = PorterStemmer()

def preprocessing_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'<.*?>', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)  # Keep numbers, remove special characters
    words = text.split()  # Tokenize text
    
    # Remove stopwords and short words
    words = [word for word in words if word not in stop_words ]

    # Apply stemming
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)  # Return processed text as a string

# Example usage
sample_text = "Breaking News!! AI is transforming the world in 2024. Check out <a href='example.com'>this</a> article!"
print(preprocessing_text(sample_text))


break news ai transform world 2024 check articl


In [19]:
news_data.columns

Index(['Text', 'label'], dtype='object')

In [22]:
news_data['Text'] = news_data['Text'].apply(lambda x:preprocessing_text(x))

In [23]:
news_data.head()

,Text,label
0,top trump surrog brutal stab back pathet video...,Fake
1,u conserv leader optimist common ground health...,Real
2,trump propos u tax overhaul stir concern defic...,Real
3,court forc ohio allow million illeg purg voter...,Fake
4,democrat say trump agre work immigr bill wall ...,Real


In [25]:
news_data['Text'][0]

'top trump surrog brutal stab back pathet video look though republican presidenti candid donald trump lose support even within rank know thing get bad even top surrog start turn exactli happen fox news newt gingrich call trump pathet gingrich know trump need keep focu hillari clinton even remot want chanc defeat howev trump hurt feel mani republican support sexual assault women turn includ hous speaker paul ryan r wi made trump lash parti gingrich said fox news look first let say trump admir tri help much big trump littl trump littl trump frankli pathet mean mad get phone call trump refer fact paul ryan call congratul debat probabl win despit trump ego tell gingrich also ad donald trump one oppon name hillari clinton name paul ryan anybodi els trump seem realiz person mad truli worst enemi ultim lead defeat one blame watch via politico featur photo joe raedl getti imag'

In [26]:
news_data['label'].value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [27]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [28]:
le.fit(news_data['label'])
news_data['label']=le.transform(news_data['label'])

In [29]:
# Get mapping
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_mapping)

{'Fake': np.int64(0), 'Real': np.int64(1)}


In [30]:
news_data.head()

,Text,label
0,top trump surrog brutal stab back pathet video...,0
1,u conserv leader optimist common ground health...,1
2,trump propos u tax overhaul stir concern defic...,1
3,court forc ohio allow million illeg purg voter...,0
4,democrat say trump agre work immigr bill wall ...,1


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf=TfidfVectorizer(stop_words='english')

Tfidf.fit(news_data['Text'])
required_text=Tfidf.transform(news_data['Text'])

In [32]:
import pickle

# Save the TF-IDF Vectorizer
with open("tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(Tfidf, file)

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(required_text, news_data['label'],test_size=0.2,random_state=42)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
model_log= LogisticRegression()

In [35]:
model_log.fit(X_train,y_train)

LogisticRegression()

In [36]:
y_pred=model_log.predict(X_test)

In [37]:
accuracy_score(y_test,y_pred)

0.9914141414141414

In [ ]:
# Save the Logistic Regression Model
with open("Log_model.pkl", "wb") as file:
    pickle.dump(model_log, file)

print("Logistic Model saved successfully!")

Logistic Model saved successfully!


In [39]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_log, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation score:", scores.mean())

Cross-validation score: 0.9883838383838384


In [40]:
from sklearn.metrics import classification_report
y_pred = model_log.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       973
           1       0.99      0.99      0.99      1007

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980



In [41]:
from sklearn.neighbors import KNeighborsClassifier
model_knn=KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)
model_knn.fit(X_train,y_train)
y_pred=model_knn.predict(X_test)
accuracy_score(y_test,y_pred)

0.8585858585858586

In [42]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_knn, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation score:", scores.mean())

Cross-validation score: 0.8582070707070708


In [43]:
from sklearn.metrics import classification_report
y_pred = model_knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.77      0.84       973
           1       0.81      0.94      0.87      1007

    accuracy                           0.86      1980
   macro avg       0.87      0.86      0.86      1980
weighted avg       0.87      0.86      0.86      1980



In [44]:
# Save the  KNN Model
with open("KNN_model.pkl", "wb") as file:
    pickle.dump(model_knn, file)

print(" KNN Model saved successfully!")

 KNN Model saved successfully!


In [45]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.997979797979798


In [46]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_rf, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation score:", scores.mean())

Cross-validation score: 0.9962121212121211


In [47]:
from sklearn.metrics import classification_report
y_pred = model_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       973
           1       1.00      1.00      1.00      1007

    accuracy                           1.00      1980
   macro avg       1.00      1.00      1.00      1980
weighted avg       1.00      1.00      1.00      1980



In [48]:
# Save the Random Forest Model
with open("random_forest_model.pkl", "wb") as file:
    pickle.dump(model_rf, file)

print("  Random Forest Model saved successfully!")

  Random Forest Model saved successfully!


In [49]:
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))

Decision Tree Accuracy: 0.9954545454545455


In [50]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_dt, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation score:", scores.mean())

Cross-validation score: 0.9978535353535353


In [51]:
from sklearn.metrics import classification_report
y_pred = model_dt.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       973
           1       0.99      1.00      1.00      1007

    accuracy                           1.00      1980
   macro avg       1.00      1.00      1.00      1980
weighted avg       1.00      1.00      1.00      1980



In [52]:
with open("Tree_model.pkl", "wb") as file:
    pickle.dump(model_dt, file)

print("Tree Model saved successfully!")

Tree Model saved successfully!


In [53]:
from sklearn.ensemble import GradientBoostingClassifier
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)
y_pred_gb = model_gb.predict(X_test)
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))

Gradient Boosting Accuracy: 0.9974747474747475


In [54]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_gb, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation score:", scores.mean())

Cross-validation score: 0.9984848484848484


In [55]:
from sklearn.metrics import classification_report
y_pred = model_gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       973
           1       1.00      1.00      1.00      1007

    accuracy                           1.00      1980
   macro avg       1.00      1.00      1.00      1980
weighted avg       1.00      1.00      1.00      1980



In [56]:
# Save the Gradient Boosting Model
with open("Gradient_model.pkl", "wb") as file:
    pickle.dump(model_gb, file)

print(" KNN Model saved successfully!")

 KNN Model saved successfully!


In [58]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_xgb, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation score:", scores.mean())

In [ ]:
from sklearn.metrics import classification_report
y_pred = model_xgb.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Save the XGBoost Model
with open("XGBoost_model.pkl", "wb") as file:
    pickle.dump(model_xgb, file)

print("XGBoost Model saved successfully!")

In [59]:
news_data.sample(5)

,Text,label
2631,trump advis discuss whether u stay pari climat...,1
5783,black protest shut fox report live tv f ing fa...,0
8501,russia say regret u move consul closur moscow ...,1
564,u senat still track releas tax bill thursday a...,1
8028,letter editor claim hillari unfit might get pe...,0


In [61]:
news_data['Text'][2631]

'trump advis discuss whether u stay pari climat pact offici washington reuter white hous advis trump administr offici meet thursday discuss whether unit state remain pari climat agreement white hous offici said wednesday meet includ member nation econom council cabinet offici environment protect agenc administr scott pruitt energi secretari rick perri schedul 1 30 p edt 1730 gmt postpon earlier month administr expect make decis whether remain parti deal time leader group seven wealthi nation meet late may member presid donald trump inner circl divid whether stay go accord agre nearli 200 countri pari 2015 would limit planetari warm part slash carbon dioxid emiss burn fossil fuel part deal unit state trump predecessor presid barack obama commit reduc emiss 26 28 percent 2005 level 2025 2016 presidenti elect campaign trump describ climat chang hoax stifl polici improv econom growth said would consid cancel pari agreement later said open stay u got better term trump advis taken differ pos

In [62]:
news_data['Text'][5783]

'black protest shut fox report live tv f ing fabric stori video last night fox news report steve harrigan report live charlott north carolina cover polic brutal protest erupt shoot perhap fox news typic shame kind protest ran footag depict protest thug looter past harrigan got hand protest hesit confront notori racist network femal african american protest went right harrigan decid educ protest gather harrigan tri shame outsid area someon gotten shot protest toler ignor white privileg longer harrigan made mistak ask man shot 50 yard know protest went repli harrigan serv purpos right give brutal lesson racism forc said matter given moment could shot color skin said could work school car still get shot polic point fact racism still exist america continu human black white tell come harrigan tri jump protest quickli call use sarcasm crowd began gather accus network protest could get video footag make f cking fabric stori know fox news done past watch protest shut fox news featur imag scree

In [63]:
sample_texts = ['trump advis discuss whether u stay pari climat pact offici washington reuter white hous advis trump administr offici meet thursday discuss whether unit state remain pari climat agreement white hous offici said wednesday meet includ member nation econom council cabinet offici environment protect agenc administr scott pruitt energi secretari rick perri schedul 1 30 p edt 1730 gmt postpon earlier month administr expect make decis whether remain parti deal time leader group seven wealthi nation meet late may member presid donald trump inner circl divid whether stay go accord agre nearli 200 countri pari 2015 would limit planetari warm part slash carbon dioxid emiss burn fossil fuel part deal unit state trump predecessor presid barack obama commit reduc emiss 26 28 percent 2005 level 2025 2016 presidenti elect campaign trump describ climat chang hoax stifl polici improv econom growth said would consid cancel pari agreement later said open stay u got better term trump advis taken differ posit decis pruitt join trump advis steve bannon oppos pari agreement trump close advis daughter ivanka trump husband jare kushner side secretari state rex tillerson said u remain part agreement perri said would best remain renegoti term georg david bank charg intern energi issu white hous nation econom council receiv feedback publicli trade coal oil compani pari decis major coal compani includ cloud peak energi said would compani global interest u stay pact l5n1hc5nc meanwhil global compani rang bp microsoft gener mill walmart sent letter trump wednesday urg presid keep u agreement protect competit',
                'black protest shut fox report live tv f ing fabric stori video last night fox news report steve harrigan report live charlott north carolina cover polic brutal protest erupt shoot perhap fox news typic shame kind protest ran footag depict protest thug looter past harrigan got hand protest hesit confront notori racist network femal african american protest went right harrigan decid educ protest gather harrigan tri shame outsid area someon gotten shot protest toler ignor white privileg longer harrigan made mistak ask man shot 50 yard know protest went repli harrigan serv purpos right give brutal lesson racism forc said matter given moment could shot color skin said could work school car still get shot polic point fact racism still exist america continu human black white tell come harrigan tri jump protest quickli call use sarcasm crowd began gather accus network protest could get video footag make f cking fabric stori know fox news done past watch protest shut fox news featur imag screenshot']
sample_preds = model_rf.predict(Tfidf.transform(sample_texts))  # Use your vectorizer

for text, pred in zip(sample_texts, sample_preds):
    print(f"Text: {text} --> \nPredicted Label: {pred}")

Text: trump advis discuss whether u stay pari climat pact offici washington reuter white hous advis trump administr offici meet thursday discuss whether unit state remain pari climat agreement white hous offici said wednesday meet includ member nation econom council cabinet offici environment protect agenc administr scott pruitt energi secretari rick perri schedul 1 30 p edt 1730 gmt postpon earlier month administr expect make decis whether remain parti deal time leader group seven wealthi nation meet late may member presid donald trump inner circl divid whether stay go accord agre nearli 200 countri pari 2015 would limit planetari warm part slash carbon dioxid emiss burn fossil fuel part deal unit state trump predecessor presid barack obama commit reduc emiss 26 28 percent 2005 level 2025 2016 presidenti elect campaign trump describ climat chang hoax stifl polici improv econom growth said would consid cancel pari agreement later said open stay u got better term trump advis taken diffe

In [64]:
sample_texts = ["NASA's Perseverance rover successfully landed on Mars today after a seven-month journey through space. The rover, equipped with advanced scientific instruments, will explore the planet’s surface and collect samples for future study. Scientists are hopeful that the mission will uncover signs of ancient microbial life.",
                "A leaked government document reveals a secret plan to implant microchips in citizens through mandatory vaccines. Experts warn that this could lead to mass surveillance. Social media users have reported strange sensations after receiving their shots, fueling concerns about the hidden technology."]
sample_preds = model_rf.predict(Tfidf.transform(sample_texts))  # Use your vectorizer

for text, pred in zip(sample_texts, sample_preds):
    print(f"Text: {text} --> \nPredicted Label: {pred}")

Text: NASA's Perseverance rover successfully landed on Mars today after a seven-month journey through space. The rover, equipped with advanced scientific instruments, will explore the planet’s surface and collect samples for future study. Scientists are hopeful that the mission will uncover signs of ancient microbial life. --> 
Predicted Label: 1
Text: A leaked government document reveals a secret plan to implant microchips in citizens through mandatory vaccines. Experts warn that this could lead to mass surveillance. Social media users have reported strange sensations after receiving their shots, fueling concerns about the hidden technology. --> 
Predicted Label: 1


In [65]:
sample_texts = ["The World Health Organization (WHO) has officially declared the end of the global pandemic after a significant decline in cases worldwide. Governments are now shifting their focus to economic recovery and strengthening healthcare infrastructure to prevent future outbreaks.",
                "Shocking footage from the International Space Station’s live feed appears to show unidentified flying objects hovering near the station. Some conspiracy theorists believe this is clear evidence of extraterrestrial contact, while NASA has yet to release an official statement."]
sample_preds = model_rf.predict(Tfidf.transform(sample_texts))  # Use your vectorizer

for text, pred in zip(sample_texts, sample_preds):
    print(f"Text: {text} --> \nPredicted Label: {pred}")

Text: The World Health Organization (WHO) has officially declared the end of the global pandemic after a significant decline in cases worldwide. Governments are now shifting their focus to economic recovery and strengthening healthcare infrastructure to prevent future outbreaks. --> 
Predicted Label: 1
Text: Shocking footage from the International Space Station’s live feed appears to show unidentified flying objects hovering near the station. Some conspiracy theorists believe this is clear evidence of extraterrestrial contact, while NASA has yet to release an official statement. --> 
Predicted Label: 1
